In [2]:
import numpy as np
import pandas as pd

In [ ]:
from datetime import datetime

from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import re

req = Request("http://insideairbnb.com/get-the-data/")
html_page = urlopen(req)

soup = BeautifulSoup(html_page, "lxml")

links = []
for link in soup.findAll('a'):
    links.append(link.get('href'))

# print(links)


In [5]:
def getdetailsfromurl(urltext):
    components = urltext.split('/')
    country = components[3]
    if len(components) == 9:
        province = components[4]
        city = components[5]
    else:
        province = components[3]
        city = components[3]
    return {'country': country, 'province':province, 'city': city}


# print(getdetailsfromurl('http://data.insideairbnb.com/new-zealand/2022-09-12/data/listings.csv.gz'))
# getdetailsfromurl('http://data.insideairbnb.com/united-states/dc/washington-dc/2022-09-14/data/listings.csv.gz')

In [25]:
data = {}
id = 0
for link in links:
        if link:
            if '.csv.gz' in link:
                data[id]= getdetailsfromurl(link)
                id+=1
                
data = pd.DataFrame(data).T
data['country'].drop_duplicates().to_frame().reset_index().drop('index', axis =1)
data['province'].drop_duplicates().to_frame().reset_index().drop('index', axis =1)
data['city'].drop_duplicates().to_frame().reset_index().drop('index', axis =1)

,city
0,amsterdam
1,antwerp
2,asheville
3,athens
4,austin
...,...
110,winnipeg
111,zurich
112,ireland
113,malta


In [8]:
def getdatalinks(datasets = ['listings', 'calendar', 'reviews'], 
                 country = 'united-states', 
                 province = None, 
                 city = None):
    
    url_dict = {name:list() for name in datasets}
    for link in links:
        if link:
            if (not province) and (not city):
                for dataset in datasets:
                    if f'/{dataset}.csv.gz' in link and f'/{country}/' in link:
                        url_dict[dataset].append(link)
          
            if (province) and (not city):
                for dataset in datasets:
                    if f'/{dataset}.csv.gz' in link and f'/{country}/{province}/' in link:
                        url_dict[dataset].append(link)

            if province and city:
                for dataset in datasets:
                    if f'/{dataset}.csv.gz' in link and f'/{country}/{province}/{city}/' in link:
                        url_dict[dataset].append(link)
    return(url_dict)

In [9]:
country = 'united-states'
province = 'nc'
city = 'asheville'
datasets = ['listings', 'calendar', 'reviews']

getdatalinks(datasets = datasets, country = country, province=province, city= city)

{'listings': ['http://data.insideairbnb.com/united-states/nc/asheville/2022-09-14/data/listings.csv.gz'],
 'calendar': ['http://data.insideairbnb.com/united-states/nc/asheville/2022-09-14/data/calendar.csv.gz'],
 'reviews': ['http://data.insideairbnb.com/united-states/nc/asheville/2022-09-14/data/reviews.csv.gz']}

In [10]:
urls = getdatalinks(datasets = datasets, country = country, province=province, city= city)
urls

{'listings': ['http://data.insideairbnb.com/united-states/nc/asheville/2022-09-14/data/listings.csv.gz'],
 'calendar': ['http://data.insideairbnb.com/united-states/nc/asheville/2022-09-14/data/calendar.csv.gz'],
 'reviews': ['http://data.insideairbnb.com/united-states/nc/asheville/2022-09-14/data/reviews.csv.gz']}

In [11]:
listing_columns = ['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name',
       'description', 'neighborhood_overview', 'picture_url', 'host_id',
       'host_url', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'calendar_updated', 'has_availability',
       'availability_30', 'availability_60', 'availability_90',
       'availability_365', 'calendar_last_scraped', 'number_of_reviews',
       'number_of_reviews_ltm', 'number_of_reviews_l30d', 'first_review',
       'last_review', 'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'license', 'instant_bookable',
       'calculated_host_listings_count',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms', 'reviews_per_month']

calendar_columns = ['listing_id', 
                    'date', 
                    'available', 
                    'price', 
                     #'adjusted_price',
                     # 'minimum_nights', 'maximum_nights'
                     ]

reviews_columns = ['listing_id', 
                   'id', 
                   'date', 
                   'reviewer_id', 
                   'reviewer_name', 
                   'comments']

In [26]:
for url in urls['listings']:
    # match = re.search('\d{4}-\d{2}-\d{2}', url)
    # print(datetime.strptime(match.group(), '%Y-%m-%d').date())
    print(getdetailsfromurl(url))
    d = (pd.read_csv(url, usecols= listing_columns, nrows=5)
            .assign(country = getdetailsfromurl(url)['country'],
                    province = getdetailsfromurl(url)['province'],
                    city = getdetailsfromurl(url)['city'])
    )
    
for url in urls['calendar']:
    c =(pd.read_csv(url, usecols= calendar_columns, nrows=5)
    )
    
for url in urls['reviews']:
    r =pd.read_csv(url, usecols= reviews_columns, nrows=5)

{'country': 'united-states', 'province': 'nc', 'city': 'asheville'}


In [28]:
d

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,country,province,city
0,108061,https://www.airbnb.com/rooms/108061,20220914194751,2022-09-14,city scrape,Walk to stores/parks/downtown. Fenced yard/Pet...,Walk to town in ten minutes! Monthly rental $1...,"I love my neighborhood! Its friendly, easy-goi...",https://a0.muscache.com/pictures/41011975/0cdf...,320564,...,NaN,f,2,2,0,0,0.67,united-states,nc,asheville
1,155305,https://www.airbnb.com/rooms/155305,20220914194751,2022-09-14,city scrape,Cottage! BonPaul + Sharky's Hostel,<b>The space</b><br />Private cottage located ...,"We are within easy walk of pubs, breweries, mu...",https://a0.muscache.com/pictures/8880711/cf38d...,746673,...,NaN,t,7,1,2,4,2.72,united-states,nc,asheville
2,156805,https://www.airbnb.com/rooms/156805,20220914194751,2022-09-14,previous scrape,"Private Room ""Ader"" at BPS Hostel",<b>The space</b><br />Private Rooms at Bon Pau...,"Easy walk to pubs, cafes, bakery, breweries, l...",https://a0.muscache.com/pictures/23447d55-fa7e...,746673,...,NaN,t,7,1,2,4,0.50,united-states,nc,asheville
3,156926,https://www.airbnb.com/rooms/156926,20220914194751,2022-09-14,city scrape,"Mixed Dorm ""Top Bunk #1"" at BPS Hostel",This is a top bunk in the mixed dorm room<br /...,NaN,https://a0.muscache.com/pictures/98f4e655-c4d6...,746673,...,NaN,t,7,1,2,4,2.34,united-states,nc,asheville
4,197263,https://www.airbnb.com/rooms/197263,20220914194751,2022-09-14,city scrape,Tranquil Room & Private Bath,"This is a comfy, peaceful and clean room with ...",NaN,https://a0.muscache.com/pictures/miso/Hosting-...,961396,...,NaN,f,2,1,1,0,0.49,united-states,nc,asheville
